# Load data

In [1]:
%%capture
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/ng_0.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/ng_1.jsonlines.zip

!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/cyberleninka_0.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/cyberleninka_1.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/cyberleninka_2.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/cyberleninka_3.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/cyberleninka_4.jsonlines.zip

!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/habrahabr_0.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/habrahabr_1.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/habrahabr_2.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/habrahabr_3.jsonlines.zip

!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/russia_today_0.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/russia_today_1.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/russia_today_2.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/russia_today_3.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/russia_today_4.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/russia_today_5.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/russia_today_6.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/russia_today_7.jsonlines.zip

!unzip ng_0.jsonlines.zip 
!unzip ng_1.jsonlines.zip

!unzip cyberleninka_0.jsonlines.zip
!unzip cyberleninka_1.jsonlines.zip
!unzip cyberleninka_2.jsonlines.zip
!unzip cyberleninka_3.jsonlines.zip
!unzip cyberleninka_4.jsonlines.zip

!unzip habrahabr_0.jsonlines.zip
!unzip habrahabr_1.jsonlines.zip
!unzip habrahabr_2.jsonlines.zip
!unzip habrahabr_3.jsonlines.zip

!unizp russia_today_0.jsonlines.zip
!unizp russia_today_1.jsonlines.zip
!unizp russia_today_2.jsonlines.zip
!unizp russia_today_3.jsonlines.zip
!unizp russia_today_4.jsonlines.zip
!unizp russia_today_5.jsonlines.zip
!unizp russia_today_6.jsonlines.zip
!unizp russia_today_7.jsonlines.zip

# Main Code

Всю загрузку данных я убрала, но по размеру датасета видно, что использовались все данные

In [2]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 8.2MB 8.8MB/s 


In [3]:
import os
import numpy as np
import pandas as pd
from string import punctuation

from gensim import summarization
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))
punct = punctuation+'«»—…“”*№–'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
PATH_TO_DATA = './'
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA) if file.endswith('jsonlines')]
print(f"There are {len(files)} files")

data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)
print(data.shape)
data.head()

There are 11 files
(10049, 6)


,content,title,summary,url,keywords,abstract
0,"MassTransit это open source библиотека, разра...",Опыт использования MassTransit 3.0,"MassTransit это open source библиотека, разраб...",https://habrahabr.ru/post/314080/,"[.net, rabbitmq, masstransit]",NaN
1,Введение и выбор решения \r\nРано или поздно ...,Геймификация форума на движке XenForo,Введение и выбор решения\r\nРано или поздно н...,https://habrahabr.ru/company/plesk/blog/313732/,"[геймификация, xenforo, форумные движки, форум...",NaN
2,\r\nНа сегодняшний день процедура реализации ...,Кластер высокой доступности на postgresql 9.6 ...,\r\nНа сегодняшний день процедура реализации «...,https://habrahabr.ru/company/etagi/blog/314000/,"[postgresq, haproxy, pgbouncer, keepalived, re...",NaN
3,"Как часто, программируя очередную бизнес-фичу,...",Как перестать бояться и полюбить синтаксически...,"Как часто, программируя очередную бизнес-фичу,...",https://habrahabr.ru/company/knopka/blog/314030/,"[irony, .net, c#, грамматический разбор, синта...",NaN
4,"Индустрия звука, о которая была у всех на слух...",Мифы и реальность: Что нужно знать о современн...,"Индустрия звука, о которая была у всех на слух...",https://geektimes.ru/company/audiomania/blog/2...,"[аудиомания, мифы и реальность, акустика, ауди...",NaN


In [5]:
data['keywords'] = data['keywords'].apply(lambda x: [el.lower() for el in x])
data.head()

,content,title,summary,url,keywords,abstract
0,"MassTransit это open source библиотека, разра...",Опыт использования MassTransit 3.0,"MassTransit это open source библиотека, разраб...",https://habrahabr.ru/post/314080/,"[.net, rabbitmq, masstransit]",NaN
1,Введение и выбор решения \r\nРано или поздно ...,Геймификация форума на движке XenForo,Введение и выбор решения\r\nРано или поздно н...,https://habrahabr.ru/company/plesk/blog/313732/,"[геймификация, xenforo, форумные движки, форум...",NaN
2,\r\nНа сегодняшний день процедура реализации ...,Кластер высокой доступности на postgresql 9.6 ...,\r\nНа сегодняшний день процедура реализации «...,https://habrahabr.ru/company/etagi/blog/314000/,"[postgresq, haproxy, pgbouncer, keepalived, re...",NaN
3,"Как часто, программируя очередную бизнес-фичу,...",Как перестать бояться и полюбить синтаксически...,"Как часто, программируя очередную бизнес-фичу,...",https://habrahabr.ru/company/knopka/blog/314030/,"[irony, .net, c#, грамматический разбор, синта...",NaN
4,"Индустрия звука, о которая была у всех на слух...",Мифы и реальность: Что нужно знать о современн...,"Индустрия звука, о которая была у всех на слух...",https://geektimes.ru/company/audiomania/blog/2...,"[аудиомания, мифы и реальность, акустика, ауди...",NaN


In [6]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

In [7]:
# to check that everything is ok
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Baseline Solution 

In [8]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [9]:
data['content_norm'] = data['content'].apply(normalize)
data['title_norm'] = data['title'].apply(normalize)
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [11]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2)
tfidf.fit(data['content_norm_str'])
texts_vectors = tfidf.transform(data['content_norm_str'])

In [12]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [13]:
evaluate(data['keywords'], keywords)

Precision -  0.07
Recall -  0.13
F1 -  0.08
Jaccard -  0.05


# First Improvement - Normalization; 

В выборке примеров можно заметить, что ключевыми словами могут быть в основном существительные и прилагательные. Попробуем добавить соответствующее условие в функцию нормализации. 

In [14]:
def normalize(text):
    
    accepted_pos = ['NOUN', 'ADJF']
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS in accepted_pos]

    return words

In [15]:
data['content_norm_v2'] = data['content'].apply(normalize)
data['title_norm_v2'] = data['title'].apply(normalize)
data['content_norm_str_v2'] = data['content_norm_v2'].apply(' '.join)

In [16]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2)
tfidf.fit(data['content_norm_str_v2'])
texts_vectors = tfidf.transform(data['content_norm_str_v2'])

In [17]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [18]:
evaluate(data['keywords'], keywords)

Precision -  0.07
Recall -  0.13
F1 -  0.08
Jaccard -  0.05


Попробуем не фильтровать нормализацию в зависимости от частей речи, оставим все  части речи: 

In [19]:
def normalize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words]
    return words

In [20]:
data['content_norm_v3'] = data['content'].apply(normalize)
data['title_norm_v3'] = data['title'].apply(normalize)
data['content_norm_str_v3'] = data['content_norm_v3'].apply(' '.join)

In [21]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2)
tfidf.fit(data['content_norm_str_v3'])
texts_vectors = tfidf.transform(data['content_norm_str_v3'])

In [22]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [23]:
evaluate(data['keywords'], keywords)

Precision -  0.08
Recall -  0.17
F1 -  0.11
Jaccard -  0.06


**Бейзлайн побит**

И  попробуем сделать следующее: У нас есть набор ключевых слов, удалим все слова из набора, если они встречаются в словосочетаниях из этого набора. Например, ключевые слова: форум, инновации, технологический форум. Удалим "форум" т.к. он встречается в словосочетани "технологический форум"

In [24]:
import copy
new_keywords = []

for list_keywords in keywords:
    to_new_keywords = []
    for keyword in list_keywords:
        _list_keywords = copy.deepcopy(list_keywords)
        _list_keywords.remove(keyword)
        presented = False
        for _keyword in _list_keywords:
            if keyword in _keyword:
                presented = True
        if presented is False:
            to_new_keywords.append(keyword)
    to_new_keywords = list(set(to_new_keywords))
    new_keywords.append(to_new_keywords)

In [25]:
evaluate(data['keywords'], new_keywords)

Precision -  0.08
Recall -  0.12
F1 -  0.09
Jaccard -  0.05


# Second Imrovement - Concatenate Title + Text

В названиях статей собирается выжимка всего текста. Попробуем учитывать также Title там где он есть. Используем при этом нормализацию бейзлайна, чтобы сравнить качество извлеченных ключевых слов именно с ним.

In [29]:
data['title_norm_str'] = data['title_norm'].apply(' '.join)
data['content_title'] = data['content_norm_str'] + data['title_norm_str']

In [30]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2)
tfidf.fit(data['content_title'])
texts_vectors = tfidf.transform(data['content_title'])

In [31]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [32]:
evaluate(data['keywords'], keywords)

Precision -  0.07
Recall -  0.13
F1 -  0.09
Jaccard -  0.05


**Бейзлайн побит (F1 Score метрика))** Посмотрим также на результаты для v3

In [33]:
data['title_norm_str_v3'] = data['title_norm_v3'].apply(' '.join)
data['content_title_v3'] = data['content_norm_str_v3'] + data['title_norm_str_v3']
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2)
tfidf.fit(data['content_title_v3'])
texts_vectors = tfidf.transform(data['content_title_v3'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])
evaluate(data['keywords'], keywords)

Precision -  0.09
Recall -  0.17
F1 -  0.11
Jaccard -  0.06


Тут улучшается Precision

# Third Improvement - Exploring TfidfVectorizer

Попробуем покрутить параметры Tf-Idf векторайзера. Занизим параметр max_df, чтобы не учитывать самые встречаемые слова.

In [51]:
tfidf = TfidfVectorizer(ngram_range=(1,3), min_df=1, max_df=0.3)
tfidf.fit(data['content_norm_str'])
texts_vectors = tfidf.transform(data['content_norm_str'])

In [52]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [53]:
evaluate(data['keywords'], keywords)

Precision -  0.07
Recall -  0.13
F1 -  0.08
Jaccard -  0.05


# Gensim 

In [54]:
_keywords =  data['content_norm_str'].apply(lambda x: summarization.keywords(x).split('\n')[:10])

In [55]:
evaluate(data['keywords'], _keywords)

Precision -  0.03
Recall -  0.05
F1 -  0.03
Jaccard -  0.02


Совсем неудачный подход